In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
#this assignment I worked with classmate and used GPT as well

## Problem 1

In [3]:
def BScallprice(sigma, S, rGrow, r, T, K):
    F= S*np.exp(rGrow*T)
    sd = sigma*np.sqrt(T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))

In [4]:
class MultiGBM:

    def __init__(self,S0,r,correlations,sigma):
        self.S0 = S0
        self.r = r
        self.correlations = correlations
        self.sigma = sigma

In [5]:
hw6p1dynamics = MultiGBM(S0=np.array([100,110]),r=0.05,
                         correlations = np.array([[1,0.8],[0.8,1]]),
                         sigma = np.diag([0.3, 0.2]))

In [6]:
class CallOnBasket:

    def __init__(self,K,T,weights):
        self.K = K
        self.T = T
        self.weights = weights

In [7]:
hw6p1contract=CallOnBasket(K=110,T=1.0,weights = np.array([1/2, 1/2]))

In [8]:
class MCengine:

    def __init__(self, M, antithetic, control, seed):
        self.M = M                                  # How many simulations
        self.antithetic = antithetic
        self.control = control
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_callonbasket_multiGBM(self,contract,dynamics):

        # You complete the coding of this function.
        # self.rng.multivariate_normal may be useful.
        # See documentation for numpy.random.Generator.multivariate_normal
        # as self.rng is an instance of numpy.random.Generator

        # You are not required to support the case where MC.control = MC.antithetic = True
        # (simultaneous use of control variate and antithetic)
        # But you are required to support the other 3 possible settings of MC.antithetic/MC.control
        # namely False/False, True/False, False/True.
        # (ordinary MC, antithetic without control, control without antithetic)
        
        K, T, weights = contract.K, contract.T, contract.weights
        S0, r, corr, sigma = dynamics.S0, dynamics.r, dynamics.correlations, dynamics.sigma
        
        L = np.linalg.cholesky(corr)
        X_0 = (np.ones((self.M, len(S0)))*np.log(S0)).T
        sigma_mat = (np.ones((self.M, len(S0))) * np.diag(sigma)).T
        
        dW_T = self.rng.standard_normal((len(S0), self.M))  * np.sqrt(T) 
        X_T = X_0 + (r-sigma_mat**2/2)*T + sigma.dot(L.dot(dW_T))
        
        S_T=np.exp(X_T)
        H_T= [i.sum() for i in (S_T.T * weights)]
        C_T=[max(0,i-110)*np.exp(-r*T) for i in H_T]
        
        price=np.mean(C_T)
        std=np.std(C_T) / np.sqrt(self.M)
        
        if self.antithetic:
            
            X_T_hat = X_0 + (r-sigma_mat**2/2)*T - sigma.dot(L.dot(dW_T))
            S_T_hat=np.exp(X_T_hat)
            H_T_hat= [i.sum() for i in (S_T_hat.T * weights)]
            C_T_hat=[max(0,i-110)*np.exp(-r*T) for i in H_T_hat]
            C_T = np.array(C_T)
            C_T_hat = np.array(C_T_hat)

            # Calculate the average of C_T and C_T_hat
            price = np.mean((C_T + C_T_hat) * 0.5)
            
            std=np.std((C_T_hat+C_T)*0.5)/np.sqrt(self.M)
        
        if self.control:
            sig = np.sqrt(sigma[0,0]**2 + sigma[1,1]**2 + 2*sigma[0,0]*sigma[1,1]*corr[0,1])/2
            S = (S0[0]*S0[1])**0.5
            rGrow = r + (2*sigma[0,0]*sigma[1,1]*corr[0,1] - sigma[0,0]**2 - sigma[1,1]**2)/8
            
            C_BS = BScallprice(sig, S, rGrow, r, T, K)
           
            X_T_star = [np.exp(i.sum()) for i in (X_T.T * weights)]
            C_T_hat = [np.exp(-r*T) * np.maximum(i-K, 0) for i in X_T_star]
            
            cov = np.cov(C_T, C_T_hat)
            beta = cov[0,1] / cov[1,1]
            Y_hat = C_T + beta*(C_BS - C_T_hat)
            price = np.mean(Y_hat)
            std = np.std(Y_hat) / np.sqrt(self.M)
        return price,std
        

In [9]:
hw6p1bMC=MCengine(M=10000,antithetic=False,control=False,seed=0)
(call_price_ordinary, std_err_ordinary) = hw6p1bMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_ordinary, std_err_ordinary)

10.017730826653947 0.16634240528457853


In [10]:
#Q1.c
hw6p1cMC=MCengine(M=10000,antithetic=True,control=False,seed=0)
(call_price_AV, std_err_AV) = hw6p1cMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_AV, std_err_AV)

9.931977784679844 0.09470581439484105


In [11]:
#Q1.e
hw6p1fMC=MCengine(M=10000,antithetic=False,control=True,seed=0)
(call_price_CV, std_err_CV) = hw6p1fMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_CV, std_err_CV)

9.98560977855196 0.004414504374051334


## Problem 2

In [12]:
class GBM:

    def __init__(self,sigma,r,S0):
        self.sigma = sigma
        self.r = r
        self.S0 = S0

In [13]:
hw6p2dynamics=GBM(sigma=0.2,r=0.02,S0=100)

In [14]:
class CallOption:

    def __init__(self,K,T):
        self.K=K
        self.T=T

In [15]:
hw6p2contract=CallOption(K=150,T=1)

In [16]:
class MCimportanceEngine:

    def __init__(self, M, lamb, seed):
        self.M = M                                  # How many simulations
        self.lamb = lamb                            # drift adjustment
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def price_call_GBM(self, contract,dynamics):

        # You complete the coding of this function.
        # self.rng.normal may be useful.
        # See documentation for numpy.random.Generator.normal
        # as self.rng is an instance of numpy.random.Generator
        
        
        K, T = contract.K, contract.T
        S0, r,  sigma = dynamics.S0, dynamics.r,  dynamics.sigma
        
       
        drift= r - 0.5*sigma**2 + self.lamb*sigma    
        X_0 = (np.ones((self.M, 1))*np.log(S0)).T
        
        
        dW_T = self.rng.standard_normal((1, self.M))  * np.sqrt(T) 
        
        X_T = X_0 + (drift)*T + sigma * dW_T
        
        S_T=np.exp(X_T)
        
        
        C_T = np.maximum(S_T - K, 0) * np.exp(-r * T)* np.exp(-self.lamb *dW_T - 0.5*self.lamb**2 *T)
        
        
        price=np.mean(C_T)
        std=np.std(C_T) / np.sqrt(self.M)
        
        return(price, std)


In [17]:
hw6p2aMC=MCimportanceEngine(M=100000,lamb=0,seed=0) #zero drift adjustment gives ordinary MC

(call_price_ordinary, std_err_ordinary) =  hw6p2aMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_ordinary, std_err_ordinary)

0.25270332833609405 0.007609255246434601


In [20]:
#Q2.b
ES_T = 165
S_0 = 100
r = 0.02
T = 1
K = 150
sigma=0.2
lamb= (np.log(ES_T/S_0)/T - r ) / sigma
print(lamb)

2.4038764395624455


In [21]:

#Q2.c
hw6p2cMC=MCimportanceEngine(M=100000, lamb= lamb ,seed=0) # Fill in the lamb parameter with the lambda that you compute in (b) 
(call_price_importsamp, std_err_importsamp) =  hw6p2cMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_importsamp, std_err_importsamp)

0.24843662621391432 0.0007734233296681479
